In [1]:
from PIL import Image
import os
import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image


### Función que permite hacer un reshape del tamaño de las imagenes a 224*224

In [ ]:

def resize_images(folder_path, new_size=(224, 224)):
    # Obtener la lista de archivos en la carpeta
    files = os.listdir(folder_path)
    output_dir = 'Osteoartritis/Processing/Valid/Ostheo'
    # Inicializar contador para el número en el nombre de la imagen
    num = 1
    
    # Iterar sobre cada archivo en la carpeta
    for file_name in files:
        # Verificar si el archivo es una imagen
        if file_name.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            # Construir la ruta completa del archivo
            file_path = os.path.join(folder_path, file_name)
            
            # Abrir la imagen
            img = Image.open(file_path)
            
            # Cambiar el tamaño de la imagen
            resized_img = img.resize(new_size)
            
            # Construir el nuevo nombre de la imagen redimensionada
            new_name = f"resized_osteortritis_{num}"
            
            # Guardar la imagen redimensionada con el nuevo nombre
            resized_img.save(os.path.join(output_dir, new_name + ".png"))
            
            # Incrementar el contador
            num += 1

# Ruta de la carpeta que contiene las imágenes
folder_path = "Osteoartritis/Dataset/Valid/Valid/Osteoarthritis"

# Cambiar el tamaño de las imágenes en la carpeta y cambiarles el nombre
resize_images(folder_path)

## Data Augmentation using Pytorch

###### Función para agregar ruido gaussiano a las imagenes

In [ ]:
def add_gaussian_noise(img):
    
    assert isinstance(img, torch.Tensor)
    dtype = img.dtype
    if not img.is_floating_point():
        img = img.to(torch.float32)
    
    # Genera un valor aleatorio sigma entre 0 y 0.1
    sigma = torch.rand(1) * 1e-5
    
    out = img + sigma * torch.randn_like(img)
    
    if out.dtype != dtype:
        out = out.to(dtype)
        
    return out



###### Definición de transformacion

In [ ]:

transform_img = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ColorJitter(brightness=0.5),
    transforms.RandomRotation(degrees=5),
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomAffine(degrees=0, scale=(0.8, 1.2))
])


In [ ]:
img =  Image.open("Osteoartritis/Processing/Train/Ostheo/resized_osteortritis_5.png")
transform_img(img)

##### Función que permite hacer data augmentation, pasando como parametro la direccion de la carpeta donde se encuentran las imagenes originales y la direccion donde se van a guardar las nuevas imagenes sinteticas. Se generan 3 imagenes sinteticas por cada imagen original

In [10]:

def data_augmentation(folder_path, output_path):
    # Obtener la lista de archivos en la carpeta
    files = os.listdir(folder_path)

    # Inicializar contador para el número en el nombre de la imagen
    num = 1

    # Definir el transformador
    transform_img = transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.ColorJitter(brightness=0.5),
        transforms.RandomRotation(degrees=5),
        transforms.RandomHorizontalFlip(p=0.2),
        transforms.RandomAffine(degrees=0, scale=(0.8, 1.2))
    ])

    # Iterar sobre cada archivo en la carpeta
    for file_name in files:
        # Verificar si el archivo es una imagen
        if file_name.endswith(('.jpg', '.jpeg', '.png', '.gif')):
            # Construir la ruta completa del archivo
            file_path = os.path.join(folder_path, file_name)

            # Abrir la imagen
            img = Image.open(file_path)
            # Aplicar transformaciones
  
            transformed_image = transform_img(img)

            # Construir el nuevo nombre de la imagen
            new_name = f"DAug_osteoartritis_{num}"

            # Guardar la imagen transformada con el nuevo nombre
            transformed_image.save(os.path.join(output_path, new_name + ".png"))


            # Incrementar el contador
            num += 1
            

In [11]:
# Ruta de la carpeta que contiene las imágenes de Train - Osteoartritis
folder_path = "Osteoartritis/Processing/Train/Ostheo"
out_path = "Osteoartritis/Processing/Data_augmentation_train/Osteo"
data_augmentation(folder_path,out_path)

In [ ]:
# Ruta de la carpeta que contiene las imágenes de Train - Normal
folder_path = "Osteoartritis/Processing/Train/Normal"
out_path = "Osteoartritis/Processing/Data_augmentation_train/Normal"
data_augmentation(folder_path,out_path)